In [18]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
import os


In [19]:
# Define the model to use
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the model to use
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

In [63]:
email = """I want information on getting a compost bin. I have submitted case number: 9578014."""

# Define the individual chains that make up each step of the complex workflow

### Define the sentiment_chain

In [61]:
# Define a sentiment prompt that returns the given email and a sentiment response
sentiment_prompt = PromptTemplate(
    input_variables=["email"],
    output_key="sentiment",
    template="""
    Given the following email text:
    {email}

    Please provide a sentiment analysis with a response value of 'Negative', 'Neutral', or 'Positive'.

    Format:
    <your sentiment here>
    """
)
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt, output_key="sentiment")

In [49]:
# Test the sentiment chain
result_1 = sentiment_chain.run(email=email)
result_1

'sentiment: Neutral'

### Define the topic_chain

In [50]:
# Define the topics and their descriptions
topics_dict = {
    'Homeless': 'words similar to homeless, shelter,encampment',
    'Graffiti': 'words similar to graffiti, paint, tagging',
    'Pothole': 'words similar to pothole, holes',
    'Animal': 'words similar to animal, pest, pets, barking',
    'Vegitation': 'words similar to weeds, trees, limbs, overgrown',
    'Neighborhood': 'words similar to HOA, RNO, sidewalk, fence',
    'Snow Removal': 'words similar to snow, ice, plows',
    'Vehicle': 'words similar to vehicle, car, motorcycle, automobile',
    'Parking': 'words similar to parking',
    'Police': 'words similar to police, gang, loud, drugs, crime',
    'Fireworks': 'words similar to fireworks',
    'Dumping': 'words similar to dumping',
    'Trash': 'words similar to trash, garbage, compost, recycling',
    'Housing': 'words similar to rent, rental, apartments, housing',
    'Policy': 'words similar to policy, tax, taxes, mayor, council, councilwoman, councilman, environmental, environment, rezoning, rezone, government, politics',
    'Street Racing': 'words similar to racing',
    'Transit': 'words similar to transit, traffic, pedestrian, intersection, bicycle, bike, speed, pavement',
    'Parks': 'words similar to park, playground, trails, pool, gym, medians',
}

# Convert the dictionary to a string
topic_descriptions = "\n".join([f"{key}: {desc}" for key, desc in topics_dict.items()])

In [59]:
# Define the prompt template
topic_prompt = PromptTemplate(
    input_variables=["email", "sentiment"],
    output_key="topics",
    template=f"""
    You are an email classification assistant. Based on the content of the email, please classify it into one or more of the following topics:

    {topic_descriptions}

    Email content:
    {{email}}

    Please list the relevant topics based on the email content. If multiple topics apply, separate them with commas. Only return the topic names.
    If you do not know the topic, return 'Other'.

    Format:
    <your topics here>
    """
)
topic_chain = LLMChain(llm=llm, prompt=topic_prompt, output_key="topics")

In [62]:
# Test the sentiment and topic chains
sequential_chain = SequentialChain(
    chains=[sentiment_chain, topic_chain],
    input_variables=["email"],  # Initial input needed
    output_variables=["email", "sentiment", "topics"]  # Final output of the chain
)
result_2 = sequential_chain({"email": email})
print(result_2)


{'email': 'I want information on getting a compost bim. I have submitted case number: 9578014.', 'sentiment': 'Neutral', 'topics': 'Trash'}


### Define the rag_chain

In [ ]:
# Define a function to format the documents retrieved from the vector store
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Specify the embeddings and directory where the vector store database is located
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
persist_directory = "../chroma_db"

# Load the saved vectorstore
vector_store = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)

# Retrieve and generate using the relevant snippets of the vector_store.
retriever = vector_store.as_retriever()

# Use a standard propmt template to perform simple queries on the loaded vectorstore
prompt = hub.pull("rlm/rag-prompt")
# [HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

# Instantiate the LLM to use
llm = ChatOpenAI(model="gpt-4", temperature=1)

# Define the chain
rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(prompt)

### Define the general_response_chain

### Define the specific_311_topic_chain

### Define the fun_fact_chain